In [1]:
# 新闻聚类

In [2]:
!pip3 install sentence-transformers scikit-learn langchain unstructured

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
# 手动下载nltk punkt、taggers，unstructured包需要
!apt-get update
!apt-get install unzip
!unzip punkt.zip
!unzip averaged_perceptron_tagger.zip
!mkdir /root/nltk_data/
!mkdir /root/nltk_data/tokenizers
!mkdir /root/nltk_data/taggers
!cp -r punkt/ /root/nltk_data/tokenizers
!cp -r averaged_perceptron_tagger/ /root/nltk_data/taggers


获取:1 https://mirrors.aliyun.com/ubuntu focal InRelease [265 kB]
获取:2 https://mirrors.aliyun.com/ubuntu focal-security InRelease [114 kB]       
获取:3 https://mirrors.aliyun.com/ubuntu focal-updates InRelease [114 kB]        
获取:4 https://mirrors.aliyun.com/ubuntu focal-backports InRelease [108 kB]      
获取:5 https://mirrors.aliyun.com/ubuntu focal/universe amd64 Packages [11.3 MB] 
获取:6 https://mirrors.aliyun.com/ubuntu focal/main amd64 Packages [1,275 kB]    
获取:7 https://mirrors.aliyun.com/ubuntu focal/multiverse amd64 Packages [177 kB]
获取:8 https://mirrors.aliyun.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
获取:9 https://mirrors.aliyun.com/ubuntu focal-security/universe amd64 Packages [1,148 kB]
获取:10 https://mirrors.aliyun.com/ubuntu focal-security/multiverse amd64 Packages [29.3 kB]
获取:11 https://mirrors.aliyun.com/ubuntu focal-security/main amd64 Packages [3,245 kB]
获取:12 https://mirrors.aliyun.com/ubuntu focal-security/restricted amd64 Packages [3,079 kB]
获取:13 https://mir

In [6]:
# 解析url结果
from langchain.document_loaders import UnstructuredURLLoader

urls = {
    '36kr': 'https://mp.weixin.qq.com/s/bz8hnmaX6tdbrmADELbrzQ',
    'tencent': 'https://mp.weixin.qq.com/s/wvwIBrN6hczhiMIicK0SnA',
    'bang': 'https://mp.weixin.qq.com/s/Gn6NZ9UtTF6Qy2SCOFf-KA'
}

def parse_url(url):
    res = []
    loader = UnstructuredURLLoader([url], mode='elements', show_progress_bar=True)
    loader.requests_kwargs = {'verify':False}
    elements = loader.load()

    for element in elements:
        res.append(element.page_content)
    return res

parse_texts = {}
for source in urls:
    source_url = urls[source]
    parse_texts[source] = parse_url(source_url)

100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


In [10]:
# 从ms上下载bge模型
from modelscope import snapshot_download
from sentence_transformers import SentenceTransformer

model_path = snapshot_download('Xorbits/bge-large-zh-v1.5', cache_dir='bge')
model = SentenceTransformer(model_path)

2023-12-11 13:25:42,046 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-12-11 13:25:42,050 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-12-11 13:25:42,051 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-12-11 13:25:42,051 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-12-11 13:25:42,123 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 569b36a1f8cf6226713a963a2d3762a4 and a total number of 945 components indexed
/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-11 13:25:42,918 - modelscope - WARNING - Model revision not specified, use revision: v0.0.1
Downloading: 100%|██████████| 191/191 [0

In [58]:
# 利用embedding，聚合title、passage
import copy
def segment(texts):
    # 核心功能：list of text -> [title, passage1, passage2]
    # 核心思想：对同一新闻来说，passage与title的相似度较大
    news = []
    one_new = []
    title_embedding = None
    
    for text in texts:
        text_embedding = model.encode(text, normalize_embeddings=True)
        if title_embedding is None:
            one_new.append(text)
            title_embedding = text_embedding
        else:
            if (title_embedding @ text_embedding.T) > 0.5:
                one_new.append(text)
            else:
                news.append(copy.deepcopy(one_new))
                one_new = [text]
                title_embedding = text_embedding

    return news

In [86]:
# 进行切分，得到(title, passage)
segments = {}
for source in parse_texts:
    segs = segment(parse_texts[source])
    if source == '36kr':
        segs = segs[3:-8]
        segs = [seg for seg in segs if len(seg) > 1]
    elif source == 'tencent':
        segs = segs[2:-6]
        segs = [seg for seg in segs if len(seg) > 1 and (len(seg[1]) - len(seg[0])) > 10]
    elif source == 'bang':
        segs = [seg for seg in segs if seg[0][0]=='【']
    segments[source] = segs

In [183]:
# segments['tencent']

In [149]:
# 开始进行clustering【针对title进行clustering】
index2new = {}
new_titles_embeddings = []
index = 0
for source in segments:
    segs = segments[source]
    for seg in segs:
        title = seg[0]
        passage = '\n'.join(seg[1:])
        new_titles_embeddings.append(model.encode(title, normalize_embeddings=True))
        index2new[index] = [source, title, passage]
        index += 1

In [199]:
from sklearn.cluster import KMeans, DBSCAN
import numpy as np

X = np.array(new_titles_embeddings)
clustering = DBSCAN(eps=0.3, min_samples=2, metric='cosine').fit(X)

In [200]:
# 解析聚类结果
clustering_res = {}
for i, label in enumerate(clustering.labels_):
    if label not in clustering_res:
        clustering_res[label] = [index2new[i]]
    else:
        clustering_res[label].append(index2new[i])

In [201]:
# 可视化聚类结果
for label in clustering_res:
    if label == -1:
        # noisy points（未聚类的点），因此不展示
        continue
    print(f'cluster {label}:')
    print()
    for new in clustering_res[label]:
        print(f'from:{new[0]}\ntitle:{new[1]}\npassage:{new[2]}')
        print()
    print('#' * 30)
    print()

cluster 0:

from:36kr
title:小米辟谣汽车发布会定档12月28日：有考虑这个时间，其他内容全是假的
passage:小红书博主“电玩科技”9日发文，称小米内部已经“敲定”28日在北京举行小米汽车亮相发布会，将主要描述新车外观、内饰设计、性能操控、智能座舱及智能驾驶等方面的卖点，届时还会公布新车的预售价格。同时，雷军已经多次检查了现场用到的 PPT，并将亲自进行演讲。对此，汽车博主“电动扒士”联系小米汽车内部工作人员，对方表示：“时间还没定，确实在考虑这个时间点，其他内容全是假的。”（IT之家）

from:bang
title:【小米辟谣汽车发布会定档12月28日：有考虑这个时间，其他内容全是假的】
passage:小红书博主“电玩科技”9日发文，称小米内部已经“敲定”28日在北京举行小米汽车亮相发布会，将主要描述新车外观、内饰设计、性能操控、智能座舱及智能驾驶等方面的卖点，届时还会公布新车的预售价格。同时，雷军已经多次检查了现场用到的 PPT，并将亲自进行演讲。对此，汽车博主@电动扒士联系小米汽车内部工作人员，对方表示：“时间还没定，确实在考虑这个时间点，其他内容全是假的。”（IT之家）

##############################

cluster 1:

from:36kr
title:元旦春节出行预订大幅上升，有平台酒店预订量同比增5倍
passage:元旦假期的整体旅游预订量同比增长4倍，其中机票订单同比增长4倍，酒店预订量同比增长5倍。根据在线旅游平台数据显示，截至目前，元旦假期旅游热度同比上升181%。进入12月以来，元旦假期出行的旅游产品预订量环比增长57%，玩雪和避寒成为假期出行的两大主题。(央视新闻)

from:bang
title:【元旦春节出行预订大幅上升，有平台酒店预订量同比增5倍】
passage:虽然距离2024年元旦春节还有一段时间，但是对假期出行的关注以及相关预订已经开始悄然升温。记者从在线旅游预订平台了解到，元旦假期的整体旅游预订量同比增长4倍，其中机票订单同比增长4倍，酒店预订量同比增长5倍。根据在线旅游平台数据显示，截至目前，元旦假期旅游热度同比上升181%。进入12月以来，元旦假期出行的旅游产品预订量环比增长57%，玩雪和避寒成为假期出行的两大主题。（财联社）

########